In [2]:
import pandas as pd
import pickle
import glob
    

## Import txt files

In [2]:
dataDir = 'C:/Users/joel0921/OneDrive - IVL Svenska Miljöinstitutet AB/DigiDrick II/VIVAB/UF_EXPORT_Raw_200401/'
filenames = glob.glob(dataDir + '*.txt') 
count=1
signals = []
for file in filenames:
    print('Importing file ' + str(count) + ' out of ' + str(len(filenames)))
    sensor_data = pd.read_csv(file, delimiter='\t')
    col = sensor_data.index
    index = []
    value = []
    for ix in range(len(col)):
        index.append(col[:][ix][0])
        value.append(col[:][ix][1])
            
    key = {'JAN':'01', 'FEB':'02', 'MAR':'03', 'APR':'04', 'MAJ':'05', 'JUN':'06', 'JUL':'07', 'AUG':'08', 'SEP':'09', 'OKT':'10', 'NOV':'11', 'DEC':'12'}
    index_mod = []
    for date in index:
        for month, num in key.items():
            date = date.replace(month, num)
        index_mod.append(date)

    index_dt = pd.to_datetime(index_mod, dayfirst=True)
    signals.append(pd.DataFrame(data=value, index=index_dt, columns=[sensor_data.columns[0]]))
    count += 1

    
txt_data = pd.DataFrame()
for s in signals:
    txt_data = txt_data.join(s, how='outer')

Importing file 1 out of 72
Importing file 2 out of 72
Importing file 3 out of 72
Importing file 4 out of 72
Importing file 5 out of 72
Importing file 6 out of 72
Importing file 7 out of 72
Importing file 8 out of 72
Importing file 9 out of 72
Importing file 10 out of 72
Importing file 11 out of 72
Importing file 12 out of 72
Importing file 13 out of 72
Importing file 14 out of 72
Importing file 15 out of 72
Importing file 16 out of 72
Importing file 17 out of 72
Importing file 18 out of 72
Importing file 19 out of 72
Importing file 20 out of 72
Importing file 21 out of 72
Importing file 22 out of 72
Importing file 23 out of 72
Importing file 24 out of 72
Importing file 25 out of 72
Importing file 26 out of 72
Importing file 27 out of 72
Importing file 28 out of 72
Importing file 29 out of 72
Importing file 30 out of 72
Importing file 31 out of 72
Importing file 32 out of 72
Importing file 33 out of 72
Importing file 34 out of 72
Importing file 35 out of 72
Importing file 36 out of 72
I

## Import Excel-file

In [4]:
data_file = 'C:/Users/joel0921/OneDrive - IVL Svenska Miljöinstitutet AB/DigiDrick II/VIVAB/UF-fullskaleanläggning_processignaler_ja.2018-april2019.xlsx'

xl = pd.ExcelFile(data_file)
sheet_names = xl.sheet_names
excel_data = pd.DataFrame()
for sheet in sheet_names[1:]:
    print('Importing: ' + sheet)
    df = pd.read_excel(xl, sheet_name=sheet)  # Import data file to pandas Dataframe
    df.index = df['Tid']
    df = df.drop(['Tid'], axis=1)
    excel_data = pd.concat([excel_data, df])

excel_data = excel_data.drop_duplicates()
# input_data.to_pickle('data.pkl')

Importing: Janurari 2018
Importing: Februari 2018
Importing: Mars 2018
Importing: April 2018
Importing: Maj 2018
Importing: Juni 2018
Importing: Juli 2018
Importing: Augusti 2018
Importing: September 2018
Importing: Oktober 2018
Importing: November 2018
Importing: December 2018
Importing: Januari 2019
Importing: Februari 2019
Importing: Mars 2019
Importing: April 2019


## Merge data

In [5]:
joined_data = pd.concat([excel_data, txt_data])
joined_data_resampled = joined_data.resample('1T').median().ffill()
joined_data_resampled.to_pickle('joined_data_180101-200401_1m.pkl')

C:\Users\joel0921\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


## Split data into <10 MB pieces

In [6]:
large_data = pd.read_pickle('joined_data_180101-200401_1m.pkl')
num_pcs = round(len(large_data.index) / 80)
for ix in range(80-1):
    small_data = large_data.iloc[ix*num_pcs + 1:(ix+1)*num_pcs]
    small_data.to_pickle('data/data' + str(ix) + '.pkl')
    
large_data.iloc[80*num_pcs + 1:].to_pickle('data/data80.pkl')

14785
                     V-VT-5751RAG-PART-GRUND  \
2018-01-01 00:01:00                      NaN   
2018-01-01 00:02:00                      NaN   
2018-01-01 00:03:00                      NaN   
2018-01-01 00:04:00                      NaN   
2018-01-01 00:05:00                      NaN   
...                                      ...   
2018-01-11 06:20:00                      NaN   
2018-01-11 06:21:00                      NaN   
2018-01-11 06:22:00                      NaN   
2018-01-11 06:23:00                      NaN   
2018-01-11 06:24:00                      NaN   

                     V-VV-1051KVG-UF-KALK_ENERGI_M3_VISA  \
2018-01-01 00:01:00                                  NaN   
2018-01-01 00:02:00                                  NaN   
2018-01-01 00:03:00                                  NaN   
2018-01-01 00:04:00                                  NaN   
2018-01-01 00:05:00                                  NaN   
...                                                  ... 

                     V-VT-5751RAG-PART-GRUND  \
2018-01-11 06:26:00                      NaN   
2018-01-11 06:27:00                      NaN   
2018-01-11 06:28:00                      NaN   
2018-01-11 06:29:00                      NaN   
2018-01-11 06:30:00                      NaN   
...                                      ...   
2018-01-21 12:45:00                      NaN   
2018-01-21 12:46:00                      NaN   
2018-01-21 12:47:00                      NaN   
2018-01-21 12:48:00                      NaN   
2018-01-21 12:49:00                      NaN   

                     V-VV-1051KVG-UF-KALK_ENERGI_M3_VISA  \
2018-01-11 06:26:00                                  NaN   
2018-01-11 06:27:00                                  NaN   
2018-01-11 06:28:00                                  NaN   
2018-01-11 06:29:00                                  NaN   
2018-01-11 06:30:00                                  NaN   
...                                                  ...   
201

                     V-VT-5751RAG-PART-GRUND  \
2018-02-21 08:06:00                19.040001   
2018-02-21 08:07:00                19.040001   
2018-02-21 08:08:00                19.040001   
2018-02-21 08:09:00                19.040001   
2018-02-21 08:10:00                19.040001   
...                                      ...   
2018-03-03 14:25:00                16.280001   
2018-03-03 14:26:00                16.280001   
2018-03-03 14:27:00                16.280001   
2018-03-03 14:28:00                16.280001   
2018-03-03 14:29:00                16.280001   

                     V-VV-1051KVG-UF-KALK_ENERGI_M3_VISA  \
2018-02-21 08:06:00                                  NaN   
2018-02-21 08:07:00                                  NaN   
2018-02-21 08:08:00                                  NaN   
2018-02-21 08:09:00                                  NaN   
2018-02-21 08:10:00                                  NaN   
...                                                  ...   
201

[14784 rows x 83 columns]
                     V-VT-5751RAG-PART-GRUND  \
2018-04-13 16:11:00                19.139999   
2018-04-13 16:12:00                19.139999   
2018-04-13 16:13:00                19.139999   
2018-04-13 16:14:00                19.139999   
2018-04-13 16:15:00                19.150000   
...                                      ...   
2018-04-23 22:30:00                17.980000   
2018-04-23 22:31:00                17.980000   
2018-04-23 22:32:00                17.980000   
2018-04-23 22:33:00                17.980000   
2018-04-23 22:34:00                17.980000   

                     V-VV-1051KVG-UF-KALK_ENERGI_M3_VISA  \
2018-04-13 16:11:00                                  NaN   
2018-04-13 16:12:00                                  NaN   
2018-04-13 16:13:00                                  NaN   
2018-04-13 16:14:00                                  NaN   
2018-04-13 16:15:00                                  NaN   
...                                  

                     V-VT-5751RAG-PART-GRUND  \
2018-05-14 11:26:00                18.670000   
2018-05-14 11:27:00                18.670000   
2018-05-14 11:28:00                18.670000   
2018-05-14 11:29:00                18.670000   
2018-05-14 11:30:00                18.670000   
...                                      ...   
2018-05-24 17:45:00                16.370001   
2018-05-24 17:46:00                16.370001   
2018-05-24 17:47:00                16.370001   
2018-05-24 17:48:00                16.370001   
2018-05-24 17:49:00                16.370001   

                     V-VV-1051KVG-UF-KALK_ENERGI_M3_VISA  \
2018-05-14 11:26:00                                  NaN   
2018-05-14 11:27:00                                  NaN   
2018-05-14 11:28:00                                  NaN   
2018-05-14 11:29:00                                  NaN   
2018-05-14 11:30:00                                  NaN   
...                                                  ...   
201

...

                     V-VT-5751RAG-PART-GRUND  \
2019-09-19 07:26:00                     13.6   
2019-09-19 07:27:00                     13.6   
2019-09-19 07:28:00                     13.6   
2019-09-19 07:29:00                     13.6   
2019-09-19 07:30:00                      0.0   
...                                      ...   
2019-09-29 13:45:00                     16.4   
2019-09-29 13:46:00                     16.4   
2019-09-29 13:47:00                     16.4   
2019-09-29 13:48:00                     16.4   
2019-09-29 13:49:00                     16.4   

                     V-VV-1051KVG-UF-KALK_ENERGI_M3_VISA  \
2019-09-19 07:26:00                                  NaN   
2019-09-19 07:27:00                                  NaN   
2019-09-19 07:28:00                                  NaN   
2019-09-19 07:29:00                                  NaN   
2019-09-19 07:30:00                                  NaN   
...                                                  ...   
201

[14784 rows x 83 columns]
                     V-VT-5751RAG-PART-GRUND  \
2019-10-30 09:06:00                     15.6   
2019-10-30 09:07:00                     15.6   
2019-10-30 09:08:00                     15.6   
2019-10-30 09:09:00                     15.6   
2019-10-30 09:10:00                     15.7   
...                                      ...   
2019-11-09 15:25:00                     15.1   
2019-11-09 15:26:00                     15.1   
2019-11-09 15:27:00                     15.1   
2019-11-09 15:28:00                     15.1   
2019-11-09 15:29:00                     15.1   

                     V-VV-1051KVG-UF-KALK_ENERGI_M3_VISA  \
2019-10-30 09:06:00                                  NaN   
2019-10-30 09:07:00                                  NaN   
2019-10-30 09:08:00                                  NaN   
2019-10-30 09:09:00                                  NaN   
2019-10-30 09:10:00                                  NaN   
...                                  

[14784 rows x 83 columns]
                     V-VT-5751RAG-PART-GRUND  \
2019-12-10 10:46:00                     17.3   
2019-12-10 10:47:00                     17.3   
2019-12-10 10:48:00                     17.3   
2019-12-10 10:49:00                     17.3   
2019-12-10 10:50:00                     17.7   
...                                      ...   
2019-12-20 17:05:00                     19.9   
2019-12-20 17:06:00                     19.9   
2019-12-20 17:07:00                     19.9   
2019-12-20 17:08:00                     19.9   
2019-12-20 17:09:00                     19.9   

                     V-VV-1051KVG-UF-KALK_ENERGI_M3_VISA  \
2019-12-10 10:46:00                                  NaN   
2019-12-10 10:47:00                                  NaN   
2019-12-10 10:48:00                                  NaN   
2019-12-10 10:49:00                                  NaN   
2019-12-10 10:50:00                                  NaN   
...                                  

                     V-VT-5751RAG-PART-GRUND  \
2020-01-10 06:01:00                     20.0   
2020-01-10 06:02:00                     20.0   
2020-01-10 06:03:00                     20.0   
2020-01-10 06:04:00                     20.0   
2020-01-10 06:05:00                     20.0   
...                                      ...   
2020-01-20 12:20:00                     19.9   
2020-01-20 12:21:00                     19.9   
2020-01-20 12:22:00                     19.9   
2020-01-20 12:23:00                     19.9   
2020-01-20 12:24:00                     19.9   

                     V-VV-1051KVG-UF-KALK_ENERGI_M3_VISA  \
2020-01-10 06:01:00                                  NaN   
2020-01-10 06:02:00                                  NaN   
2020-01-10 06:03:00                                  NaN   
2020-01-10 06:04:00                                  NaN   
2020-01-10 06:05:00                                  NaN   
...                                                  ...   
202

[14784 rows x 83 columns]
                     V-VT-5751RAG-PART-GRUND  \
2020-03-01 14:06:00                     20.0   
2020-03-01 14:07:00                     20.0   
2020-03-01 14:08:00                     20.0   
2020-03-01 14:09:00                     20.0   
2020-03-01 14:10:00                     20.0   
...                                      ...   
2020-03-11 20:25:00                     20.1   
2020-03-11 20:26:00                     20.1   
2020-03-11 20:27:00                     20.1   
2020-03-11 20:28:00                     20.1   
2020-03-11 20:29:00                     20.1   

                     V-VV-1051KVG-UF-KALK_ENERGI_M3_VISA  \
2020-03-01 14:06:00                                  NaN   
2020-03-01 14:07:00                                  NaN   
2020-03-01 14:08:00                                  NaN   
2020-03-01 14:09:00                                  NaN   
2020-03-01 14:10:00                                  NaN   
...                                  